Import Dependencies

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold,learning_curve
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,make_scorer
import pickle
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from scipy import stats
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm


Load data into a DataFrame

In [2]:
# Load the data
mb_raw_data = pd.read_csv('../../data/mann_bruker.txt', sep='\t')

# Keep only necessary columns
mb_clean_frame = mb_raw_data[['Sequence', 'm/z', 'CCS','Mass','Charge','Length']]
# Group by 'Sequence' and 'Charge', and calculate median of 'Mass' and 'CCS'
mb_clean_frame = mb_clean_frame.groupby(['Sequence', 'Charge']).agg({'Mass':'median', 'CCS':'median','Length':'median'}).reset_index()
# Perform z-score transformation
mb_clean_frame['CCS_z'] = stats.zscore(mb_clean_frame['CCS'])

# Save the mean and std for later use
ccs_mean = mb_clean_frame['CCS'].mean()
ccs_std = mb_clean_frame['CCS'].std()

# Delete the raw data frame to save memory
del mb_raw_data
# randomize data set
mb_clean_frame = mb_clean_frame.sample(frac=1, random_state=1)

Train Model

In [3]:
# Split the data into input (m/z) and output (CCS) variables
X = mb_clean_frame[['Mass', 'Charge', 'Length']]
y = mb_clean_frame['CCS_z']
# Define the number of folds
k = 4
# Number of trees
n = 50

Perform k-fold cross validation

In [4]:
start_time = time.time()

kf = KFold(n_splits=k)
mse_scores = []
median_relative_errors = []
r2_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = RandomForestRegressor(n_estimators=n, random_state=1,n_jobs=-1)

    # Train the model
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Convert predictions back to original scale
    y_pred_orig = y_pred * ccs_std + ccs_mean
    y_test_orig = y_test * ccs_std + ccs_mean


    mse = mean_squared_error(y_test_orig, y_pred_orig)
    mse_scores.append(mse)

    relative_errors = np.abs((y_pred_orig - y_test_orig) / y_test_orig)
    median_relative_error = np.median(relative_errors)
    median_relative_errors.append(median_relative_error)

    r2 = r2_score(y_test_orig, y_pred_orig)
    r2_scores.append(r2)
    # Progress update
    print("Fold:", len(mse_scores))

# Create the final model
final_model = RandomForestRegressor(n_estimators=n, random_state=1,n_jobs=-1)
final_model.fit(X, y)

print("Average Mean Squared Error:", np.mean(mse_scores))
print("Average Median Relative Error:", np.mean(median_relative_errors))
print("Average R^2 Score:", np.mean(r2_scores))
elapsed_time = time.time() - start_time
print(f'Time elapsed: {elapsed_time:.2f} seconds')

Fold: 1
Fold: 2
Fold: 3
Fold: 4
Average Mean Squared Error: 7.318639233783918e-73
Average Median Relative Error: 0.016806985984330135
Average R^2 Score: 0.9890778874242914
Time elapsed: 62.35 seconds


In [5]:

def median_relative_error(y_true, y_pred):
    return np.median(np.abs((y_pred - y_true) / y_true))

scorer = make_scorer(median_relative_error, greater_is_better=False)


In [ ]:
train_sizes = np.linspace(0.01, 1.0, 100)

model = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=4)

def median_relative_error(y_true, y_pred):
    y_pred_orig = y_pred * ccs_std + ccs_mean
    y_true_orig = y_true * ccs_std + ccs_mean
    return np.median(np.abs((y_pred_orig - y_true_orig) / y_true_orig))

median_relative_error_scores = []
R2_scores = []
data_sizes = []

for train_size in tqdm(train_sizes):
    kf = KFold(n_splits=k)
    data = mb_clean_frame.sample(frac=train_size, random_state=1)
    X = data[['Mass', 'Charge', 'Length']]
    y = data['CCS_z']
    data_sizes.append(len(data))
    median_relative_errors = []
    r2_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model = RandomForestRegressor(n_estimators=n, random_state=1,n_jobs=-1)
    
        model.fit(X_train, y_train)
    
        y_pred = model.predict(X_test)
    
        y_pred_orig = y_pred * ccs_std + ccs_mean
        y_test_orig = y_test * ccs_std + ccs_mean
    
        
        relative_errors = np.abs((y_pred_orig - y_test_orig) / y_test_orig)
        median_relative_error = np.median(relative_errors)
        median_relative_errors.append(median_relative_error)

        r2 = r2_score(y_test_orig, y_pred_orig)
        r2_scores.append(r2)
    
    median_relative_error_scores.append(np.mean(median_relative_errors))
    R2_scores.append(np.mean(r2_scores))

  6%|▌         | 6/100 [00:11<04:01,  2.57s/it]

In [ ]:
# Plot Median Relative Error with trendline
plt.figure(figsize=(8, 6), dpi=300)
plt.scatter(data_sizes, median_relative_error_scores, marker='o')
coeffs = np.polyfit(data_sizes, median_relative_error_scores, 2)  
poly = np.poly1d(coeffs)
plt.plot(sorted(data_sizes), poly(sorted(data_sizes)), color='red')  
plt.title('Median Relative Error vs Data Size')
plt.xlabel('Data Size')
plt.ylabel('Median Relative Error')
plt.grid(True)
plt.show()

# Plot R^2 Scores with trendline
plt.figure(figsize=(8, 6), dpi=300)
plt.scatter(data_sizes, R2_scores, marker='o')
coeffs = np.polyfit(data_sizes, R2_scores, 2)  
poly = np.poly1d(coeffs)
plt.plot(sorted(data_sizes), poly(sorted(data_sizes)), color='red')  
plt.title('R^2 Score vs Data Size')
plt.xlabel('Data Size')
plt.ylabel('R^2 Score')
plt.grid(True)
plt.show()


Save the model

In [ ]:
# Save the model to a file
#filename = 'random_forest'
#pickle.dump(model, open(filename, 'wb'))

In [ ]:
mb_clean_frame['Predicted CCS'] = final_model.predict(X)

# Reverse the z-score transformation
mb_clean_frame['Predicted CCS'] = mb_clean_frame['Predicted CCS'] * ccs_std + ccs_mean


In [ ]:

mb_clean_frame.plot.scatter(
    x='CCS',
    y='Predicted CCS',
    c='Length',
    cmap='winter',
    alpha=0.2,
    vmin=mb_clean_frame['Length'].min(),
    vmax=mb_clean_frame['Length'].max()
)
mb_clean_frame.plot.hexbin(
    x='CCS',
    y='Predicted CCS',
    C='Length',
    reduce_C_function=np.mean,
    gridsize=50,
    cmap='magma'
)

# Print the Spearman's correlation coefficient between the predicted and actual CCS values
print("Spearman's Correlation Coefficient:", mb_clean_frame['CCS'].corr(mb_clean_frame['Predicted CCS'], method='spearman'))
# Create a scatter plot between the percent error and length
mb_clean_frame['Percent Error'] = np.abs((mb_clean_frame['Predicted CCS'] - mb_clean_frame['CCS']) / mb_clean_frame['CCS'])
mb_clean_frame.plot.scatter(
    x='Length',
    y='Percent Error',
    alpha=0.2,
    vmin=mb_clean_frame['Length'].min(),
    vmax=mb_clean_frame['Length'].max()
)


boundaries = [0.5, 1.5, 2.5, 3.5, 4.5]
cmap = mcolors.ListedColormap(['orange', 'blue', 'green', 'red'])

norm = BoundaryNorm(boundaries, cmap.N, clip=True)

fig, ax = plt.subplots()

scatter = ax.scatter(
    x=mb_clean_frame['CCS'],
    y=mb_clean_frame['Predicted CCS'],
    c=mb_clean_frame['Charge'],
    cmap=cmap,
    norm=norm,
    alpha=0.2
)
ax.set_xlabel('Experimental CCS')
ax.set_ylabel('Predicted CCS')

cbar = plt.colorbar(scatter, ticks=[1, 2, 3, 4])
cbar.set_label('Charge')

plt.show()

In [ ]:
ccs_data = mb_clean_frame["CCS"].to_numpy()
predicted_ccs_data = mb_clean_frame["Predicted CCS"].to_numpy()


x = mb_clean_frame["CCS"]
y = mb_clean_frame["Predicted CCS"]

plt.hist2d(x, y, bins=[30,30],norm=mcolors.LogNorm())
plt.xlabel("Experimental CCS")
plt.ylabel("Predicted CCS")
plt.colorbar()
plt.show()


In [ ]:
# Get the importances from the final model
importances = final_model.feature_importances_
# Sort the feature importances in descending order
indices = np.argsort(importances)[::-1]
features = X.columns

plt.figure()
plt.title("Feature Importance")
plt.bar(range(X.shape[1]), importances[indices], color="#3070B3", align="center")
plt.xticks(range(X.shape[1]), features[indices], rotation=0)
plt.xlim([-1, X.shape[1]])
plt.ylabel('Feature Importance')

plt.show()